# HW1 布尔查询之BSBI与索引压缩

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [1]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

<span style="color:blue">WO说：本代码已上传至Github仓库：https://github.com/Hanpiyd/Information-Retrieval.git 。接下来的报告内容将以
    Markdown形式呈现，字体为蓝色（区分原有Markdown）</span>

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [2]:
import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [2]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [3]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [4]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [5]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [6]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

<span style="color:blue">对于IdMap部分需要完成的两个函数-_get_str和_get_id，处理略微不同。虽然_get_str在前，但是这里选择先完成更重要的_get_id，
先判断要查询的字符串是否是字典中的Key，如果不是则将（该字符串，字典长度）这个Key-Value对存入字典（字典长度永远不可能重复，因此很适合作为id），
同时将该字符串存入列表中（由于字典和列表的长度一定相同，因此当id作为下标时从列表中取出的一定是该字符串）；如果是，则直接利用字典的特性得到相应
的Value-id即可。而对于_get_str，直接根据前面的叙述使用id作为下标从列表取相应元素即可。</span>

In [7]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id.keys():
            self.str_to_id[s] = len(self.str_to_id)
            self.id_to_str.append(s)
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [8]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [9]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [10]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [11]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [12]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

<span style="color:blue">在本部分，我们要完成parse_block这一方法，它的功能主要是将文件中的单词流转换为（term_id,doc_id），再将得到的所有元组存入
列表作为返回值，首先，我们先初始化存放元组的列表，再将基地址和相对地址进行拼接得到绝对地址。而绝对地址下又有很多子文件，因此通过os.listdir得到
所有子文件的名字。接着，以子文件为单位，先拼接相对地址和子文件名打开文件进行数据读取，再利用之前实现的方法得到相应文件的doc_id，同时将读取到
的单词流先去掉空格再进行划分。对于得到的每个单词，先利用前面的方法取到term_id,再结合doc_id得到目标元组，接着将元组存入列表即可。最后，结束
两层循环后返回列表即可。</span>

In [13]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        result_list = []
        root_addr = os.path.join(self.data_dir, block_dir_relative)
        addr_list = os.listdir(root_addr)
        for addr in sorted(addr_list):
            with open(os.path.join(root_addr, addr), 'r') as f:
                doc_id = self.doc_id_map._get_id(os.path.join(block_dir_relative, addr))
                word_list = f.read().strip().split()
                for word in word_list:
                    term_id = self.term_id_map._get_id(word)
                    term_doc_pair = [term_id, doc_id]
                    result_list.append(term_doc_pair)
        return result_list
        ### End your code

观察函数在测试数据上是否正常运行

In [14]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [15]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[[0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 0],
 [5, 1],
 [5, 1],
 [6, 1],
 [7, 1],
 [4, 1],
 [8, 1]]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [16]:
### Begin your code

### End your code

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

<span style="color:blue">在本部分我们要完成append方法，该方法具体要完成的三步内容已经以注释的形式写在了方法中，我们只需要对其进行完成即可。
首先使用前面提供的默认方法对参数postings_list进行编码。接着打开索引文件，先将文件指针移动到文件的尾部，用tell方法取到后即为开始位置，剩余的
number_of_postings_in_list和length_in_bytes_of_postings_list直接利用len方法即可完成，将得到的三个数组进行组合就得到了作为Value的三元组。
再将term和term到三元组的Key-Value对分别存放于列表和字典，再将编码后的postings_list写入索引文件即可。</span>

In [17]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        postings_list_encoded = self.postings_encoding.encode(postings_list)
        with open(self.index_file_path,'r+b') as f:
            f.seek(0,2)
            start_position = f.tell()
            number_of_posting = len(postings_list)
            length_in_bytes = len(postings_list_encoded)
            self.terms.append(term)
            self.postings_dict[term] = (start_position, number_of_posting, length_in_bytes)
            f.write(postings_list_encoded)
        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [18]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

<span style="color:blue">我们接下来实现invert_write方法，就是将我们前面得到的存放元组的列表转换为倒排表并用append方法写入磁盘。首先根据我们的
要求先初始化一个字典，以Key:term-Value:Postings_list的方法构造。接着遍历参数列表，如果元组相应的term还没有在字典出来，则存入一个term-空列表的对，
如果已经存在，则只需要从将doc_id取出放入相应的列表即可。接着将字典按照term_id大小进行排序，再通过循环将term_id和排好序的相应postings_list借助
InvertedIndexWriter的append方法写入索引文件即可。</span>

In [19]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        result_dict = {}
        for t,d in td_pairs:
            if result_dict.get(t) is None:
                result_dict[t] = []
            result_dict[t].append(d)
        seq_key = sorted(result_dict.keys())
        for i in seq_key:
            result_dict[i] = sorted(result_dict[i])
            index.append(i, result_dict[i])
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [20]:
### Begin your code

### End your code

### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

<span style="color:blue">在本部分，主要完成_initialization_hook和__next__两个方法，前者只需要初始化一个指向开始位置（即赋值为零）的变量即可，
而后者则稍微有些复杂。对于__next__方法，我们要先判断当前指针是否已经越界，如果越界则直接抛出迭代，如果不这么写后面在merge的时候会出现错误（亲身
经历），如果不越界则继续接下来的操作：先利用指针取出相应的term，再利用term从字典中取出对应的三元组，接着利用开始位置和字节长度将postings_list从
索引文件中读出并解码，接着将指针向后移一位方便后续的处理。最后将取到的term和postings_list打包返回即可。</span>

In [21]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.curr_position_ptr = 0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        if self.curr_position_ptr < len(self.terms):
            term = self.terms[self.curr_position_ptr]
            start_potision, number_of_posting, length_of_bytes = self.postings_dict[term]
            self.index_file.seek(start_potision)
            postings_list_encoded = self.index_file.read(length_of_bytes)
            postings_list = self.postings_encoding.decode(postings_list_encoded)
            self.curr_position_ptr += 1 
            return term, postings_list
        else:
            raise StopIteration
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [22]:
### Begin your code

### End your code

> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [23]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

<span style="color:blue">对于merge方法的实现，我们先准备一个temp_term和temp_list，主要是我们之后的循环只能访问当前的term和postings_list，因此
为了判断相邻的两个term是否相等，即是否需要合并，我们需要记录之前的term和postings_list（方便term相同时合并）。利用前面介绍的heapq.merge对indices
进行处理，此时所有（term,postings_list）元组都按term从小到大排序。接着开始遍历，第一个元素是特殊情况，因为它前面没有元素，那它一定与前面不同，
此时我们只需要把它的term和postings_list记录下来即可。如果当前的term和前一个term不等还非第一个元素，说明此时的term和前面的一个（或一批）term已经
不同，这时将前面记录的term和postings_list用append方法写入索引文件，然后记录当前的term和postings_list作为新的起点。如果term和前一个的term相同
时，记录的term不需要变，而记录的postings_list和当前的postings_list先进行拼接，再利用set的特性进行去重并排序，最后再转换回列表，这里就涉及到
了记录list的第二个特性-记录当前一个（或一批）相同term的合并后的postings_list（局部结果）。循环结束后将当前记录的term和postings_list再写入索引
文件即可结束（很容易发现无论何种情况最后一个（或一批）记录的term和对应的list都不会在循环被写入，因此我们需要单独处理）。</span>

In [24]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        temp_term = None
        temp_list = None
        for term, postings_list in heapq.merge(*indices):
            if term !=temp_term:
                if temp_term is None:
                    temp_term = term
                    temp_list = postings_list
                else:
                    merged_index.append(temp_term, list(sorted(set(temp_list))) )
                    temp_term = term
                    temp_list = postings_list
            else:
                temp_list = temp_list + postings_list
        if temp_term is not None:
            merged_index.append(temp_term, list(sorted(set(temp_list))))
        ### End your code

首先确保它在测试数据上正常运行

In [25]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

接下来对整个数据集构建索引

In [25]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [28]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

<span style="color: blue">_get_postings_list方法的实现其实非常简单，还是得以与前面的工作。使用term从postings_dict我们可以取到对应的三元组，
先利用start_position将索引文件的指针移动到相应postings_list的开头，再读取出长为lengths_of_bytes的字节流，读取出postings_list后解码返回即可。
</span>

In [26]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        start_position, number_of_posting, length_of_bytes = self.postings_dict[term]
        self.index_file.seek(start_position)
        postings_list_encoded = self.index_file.read(length_of_bytes)
        postings_list = self.postings_encoding.decode(postings_list_encoded)
        return postings_list
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [27]:
### Begin your code

### End your code

现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

<span style="color: blue">对于这部分完成两个列表求交集的布尔检索算法，根据课上所学的知识可以非常简单实现。先初始化两个相应的指针和一个存放
结果的列表，接着开始循环即可，终止条件即为其中至少一个列表已遍历完毕。对于两个指针所指向的值，如果不相等的话只需要让较小的指针加一即可，当
相等的时候就将相等的值存入列表中，两个指针同时加一。最后返回已经存放了结果的列表即可。</span>

In [28]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    ptr1 = 0
    ptr2 = 0
    result_list = []
    while ptr1 < len(list1) and ptr2 < len(list2):
        if list1[ptr1] < list2[ptr2]:
            ptr1 += 1
        elif list1[ptr1] > list2[ptr2]:
            ptr2 += 1
        else:
            result_list.append(list1[ptr1])
            ptr1 += 1
            ptr2 += 1
    return result_list
    ### End your code

简单的测试样例

In [29]:
### Begin your code

### End your code

现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

<span style="color: blue">对于retrieve方法，我们就需要综合前面所实现的两个方法来进行辅助，我们先初始化一个InvertedIndexMapper帮助从索引文件
获取相应的postings_list，接着初始化一个列表用来存放以doc_id为元素的中间列表。由于查询输入的是字符串，因此我们要先使用split方法将字符串切分为
单词，接着遍历单词，如果存在一个单词并不在所有原文章中出现过，则直接返回空列表（注释规定），如果都出现过，我们只需要先将相应的单词映射为
term_id，再利用term_id使用mapper读出相应的postings_list，将所有的postings_list进行交集运算（每次都处理两个，之前处理的结果和下一个列表进行
交集，比如说第一个和第二个列表交的结果与第三个列表再进行相交运算）最后就得到了相应的结果列表。但此时的列表是以doc_id为元素的，而返回值希望
我们直接使用doc名，因此再借助IdMap将doc_id映射为doc名，最后返回以doc名为单位的列表即可。</span>

In [30]:
#%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        with InvertedIndexMapper(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as mapper:
            result = None
            term_list = query.split()
            for term in term_list:
                term_id = self.term_id_map.str_to_id.get(term)
                if term_id is None:
                    return []
                else:
                    temp_list = mapper._get_postings_list(term_id)
                    if result is None:
                        result = temp_list
                    else:
                        result = sorted_intersect(result, temp_list)
            result_str_list = []
            for i in result:
                temp_str = self.doc_id_map._get_str(i)
                result_str_list.append(temp_str)
            return result_str_list
        ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [31]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [32]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [33]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [34]:
set(my_results) - set(reference_results)

set()

In [35]:
set(reference_results) - set(my_results)

set()

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [36]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

<span style="color: blue">对于索引压缩这部分的工作，我选择使用VB和gap-encoding来实现。先讲解VB的原理：对于编码，假如有一个正整数n，先将其为二进制形式，再
按每七位划分为一组（总位数不是7的倍数则从高位侧补0），再在非最后一组的其他每组前再填充一位0，而最后一组则填1，就实现了编码。解码就为逆过程。
那么对于VB的实现，我们先涉及三个辅助方法，对于VBEncodeNumber，主要就是根据刚才的原理先将一个数n划分为7位一组，存储于列表中，由于2的7次方为128，
所以对于填充1我们只需要将对应的数加128即可，其他填充0我们不需要作额外的处理。由于我们采取的是append的尾插法来存储数据，最后我们需要将列表中的
元素位置进行反转。对于VBEncode，只是输入从一个数变成了一列表的数，只需要以此读取数据调用VBEncodeNumber再将得到的列表extend入初始化存储结果的列
表，再遍历结束后返回该列表即可。而对于VBDecode，我们得到的输入即为调用VBEncode得到的列表，列表可能包含多个数的VB编码，因此我们需要判断一个数的
相应VB编码终止于哪一个数，根据前面中延续位的知识我们可以发现，从上一个大于127的数之后的第一个数到下一个大于127的数直接的所有数都是一个数的VB编码，
我们只需要将这些数根据高低位进行一定的处理即可得到该数的原十进制版本（具体方法在VBDecode已有完成，信息检索导论教材亦有伪代码，值得注意的是最后
一个数要减去一个128，以去掉添加延续位对原数值可能造成的影响），并将其放入保存结果的列表。最后返回列表即可。
<br>
<br>
接下来，我们正式开始Encode和Decode方法的实现，在此之前先介绍一下gap-encoding的概念。在我的理解中，gap-encoding还有另一个名字-差分，即除了列表
的第一个元素外我们都保留的是原列表相应位置的项与其前一项的差值。对于Encode，我们也是先进行差分列表的求解，再使用前面实现的辅助函数对差分列表
进行VB编码即可。最后将编码后的列表以字节串的形式返回即可。而对于Decode，过程正好相反即可，我们先将字节串转回编码后的列表，再使用VBDecode解码
得到差分列表，借助差分的知识还原回原本的列表返回即可。
</span>

In [37]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def VBEncodeNumber(n):
        byte = []
        while n // 128 > 0:
            temp_num = n % 128
            byte.append(temp_num)
            n = n // 128
        byte.append(n)
        byte[0] += 128
        byte = byte[::-1]
        return byte
    
    @staticmethod
    def VBEncode(numbers):
        bytestream = []
        for num in numbers:
            temp_list = CompressedPostings.VBEncodeNumber(num)
            bytestream.extend(temp_list)
        return bytestream
    
    @staticmethod
    def VBDecode(bytestream):
        numbers = []
        n = 0
        for byte in bytestream:
            if byte >= 128:
                n = n * 128 + (byte - 128)
                numbers.append(n)
                n = 0
            else:
                n = n * 128 + byte
        return numbers
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        result_list = []
        for i in range(0, len(postings_list)):
            result_list.append(0)
        result_list[0] = postings_list[0]
        for i in range(1, len(postings_list)):
            result_list[i] = postings_list[i] - postings_list[i - 1]
        res = CompressedPostings.VBEncode(result_list)
        res_byte = array.array('B', res).tobytes()
        return res_byte
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        data = array.array('B')
        data.frombytes(encoded_postings_list)
        encoded_list = data.tolist()
        postings_list = CompressedPostings.VBDecode(encoded_list)
        result_list = []
        for i in range(0, len(postings_list)):
            result_list.append(0)
        result_list[0] = postings_list[0]
        for i in range(1, len(postings_list)):
            result_list[i] = result_list[i - 1] + postings_list[i]
        return result_list
        ### End your code

首先，如果你实现了额外的辅助函数，写一些测试样例

In [38]:
### Begin your code

### End your code

我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [39]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [40]:
### Begin your code

### End your code

现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [41]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [209]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [42]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

像之前一样，用已构造好的查询语句来测试是否正常运行

In [43]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

<span style="color: blue">对于额外编码方式这部分的工作，我选择使用gamma-encoding（具体实现方法还是参考信息检索教材）和gap-encoding。先介绍一下
gamma-encoding的实现方法，对于一个正整数n，先将其转为二进制形式，计算此时的长度L，那么n的gamma编码方式即为L-1个1 + 0 + n的二进制形式去掉最高位
（此处的 + 可以理解为字符串的拼接运算）。根据gamma-encoding的原理设计相应的辅助方法GammaEncode，具体的实现和刚才的原理相同，唯一值得注意的是Python
中将十进制数转为二进制字符串会在最前面多加两位表示二进制的0b，因此在去掉最高位的基础上还要多去掉两位。
<br>
<br>
而对于Encode函数，和索引压缩类似，我们使用相同的方法先得到输入列表的差分列表，再将差分列表的每一项使用辅助方法转化为相应编码后得到的字符串存储
在相应存放结果的列表中。由于要求使用多字节而非比特来存储倒排记录表，因此先将res列表中每个gap所对应的字符串拼接在一起，接着对字符串
进行处理，检查字符串的长度是否为8的倍数（因为1字节 = 8比特），如果不是则在最前面补0。将字符串切割为8位一组的子字符串，再将子字符串转为整型后都放入结果列表中，最后转为字节串的形式返回即可。
<br>
<br>
对于Decode函数，我们先将字节串转为相应的列表，再将列表中的每一项都转为二进制字符串并进行拼接，接着对拼接后的字符串进行gamma-encoding的解码工作。
对于解码，我们根据gamma-encoding的性质可以发现，一连串的1出现时，连续的1的位数总长即为相应编码数在字符串保留的位数，由于插入的1与原数的二进制形式
（去掉最高位）间仅间隔一个0，因此我们借助一个指针和一个记录长度的变量就可以完成将字符串解码为差分列表。指针可以帮助我们记录连续的1的起点，也可以
帮助我们定位截取字符串的开始，遇到连续的1时先获取连续的1的位数总长，再将指针当前值加上位数总长再加一（跳过0）就是相应数对于字符串的开始，截取字符串
再于最高位拼接上1最后转回十进制数就得到了相应数，当然结束后要将指针跨越相应数的二进制部分指向下一个数的部分，并将长度重新置为0。当然，值得注意的是
，我们在编码中为了实现多字节存储可能在字符串前插入了用来补位的多余的0，因此在处理前我们需要先将前导0全部去掉，不过注意特殊情况，如果前导0的数量为8，那么第一个数一定为1（因为不会直接补8个0），需要对其进行直接放入列表的操作。将得到的所有数
都存入列表就得到了差分列表，再利用差分知识计算就可以得到原始列表，将原始列表返回即可。
<br>
<br>
但是既然已经思考到这里，你肯定已经很容易想到了一个有意思的问题：如果列表第一个数是1，后面又有多个gap为1，那岂不是束手无策了？很简单，我们在编码过程中直接从2开始不就好了。对Encode和Decode函数进行修改，给每个编码前的gap都加1，在解码后得到的gap再减1，就得到了原gap了，这样就避免了这个问题。

In [44]:
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def GammaEncode(num):
        n = bin(num)[2:]
        return (len(n) - 1) * '1' + '0' + n[1:]
    
    ### End your code

    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        result_list = []
        for i in range(0, len(postings_list)):
            result_list.append(0)
        result_list[0] = postings_list[0] + 1
        for i in range(1, len(postings_list)):
            result_list[i] = postings_list[i] - postings_list[i - 1] + 1
        res = []
        for i in range(0, len(result_list)):
            res.append(ECCompressedPostings.GammaEncode(result_list[i]))
        string = None
        for i in res:
            if string is None:
                string = i
            else:
                string = string + i
        string_len = len(string) % 8
        need_num = (8 - string_len) % 8
        string = need_num * '0' + string
        byte_stream = bytearray()
        for i in range(0, len(string), 8):
            byte_chunk = int(string[i: i + 8], 2)
            byte_stream.append(byte_chunk)
        return array.array('B',byte_stream).tobytes()
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        postings_list = array.array('B')
        postings_list.frombytes(encoded_postings_list)
        postings_str = ''.join([bin(byte)[2:].zfill(8) for byte in postings_list])
        result = []
        ptr = 0
        base_str = 0
        while postings_str[base_str] == '0':
            base_str += 1
        #if base_str == 8:
        #    result.append(1)    //修改后的代码就没有1了，因此这里也不需要再对可能的1进行额外的判断了
        postings_str = postings_str[base_str:]
        while ptr < len(postings_str):
            bin_length = 0
            while ptr < len(postings_str) and postings_str[ptr] == '1':
                ptr += 1
                bin_length += 1
            ptr += 1
            if bin_length > 0:
                temp_str = postings_str[ptr: ptr + bin_length]
                res = int('1' + temp_str, 2)
                result.append(res)
                ptr += bin_length
            else:
                result.append(1)
        res_list = [result[0] - 1]
        for i in range(1, len(result)):
            res_list.append(res_list[i - 1] + result[i] - 1)
        return res_list
        ### End your code

<span style="color: blue">让我们编写一段简单的测试代码来验证一下Gamma编码的正确性吧</span>

In [47]:
postings = [1,2,3,4,5,10,45,169]
#postings = [1,3,6,10,19,32,56,567,1592]
encoded = ECCompressedPostings.encode(postings)
decoded = ECCompressedPostings.decode(encoded)

print(postings)
print(encoded)
print(decoded)

[1, 2, 3, 4, 5, 10, 45, 169]
b'\t$\x9a\xf8\x9f\xbd'
[1, 2, 3, 4, 5, 10, 45, 169]


<span style="color: blue">通过本次实验，我对课上所讲的BSBI、布尔检索以及可变长编码和Gamma编码（部分自学）在理论的基础上进行了实际的实现，
并加深了我对相应知识的理解与掌握。本次实验的难点主要在于从庞大的框架中知道需要用什么、该怎么用、怎么用更好，需要结合课上所讲述的知识，
并在作业中进行实际尝试，才能解决相应的问题。</span>

# 作业提交

本次作业用时y约3周，截止日期为9.30（国庆节前）。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到ir24fall@163.com，邮件和文件命名方式均为`学号_姓名_hw1`，如`1811412_戚晓睿_hw1.ipynb`